In [1]:
DIRECTORY = "/home/saint/datasets/competitions/tgs-salt-identification-challenge"
SAVE_PATH = "/home/saint/models/salt/"
SUBMISSIONS_PATH = "/home/saint/submissions/salt/"
WEIGHTS_PATH = "/home/saint/weights/ternaus_net_v2_deepglobe_buildings.pt"

MODEL_NAME = 'ternausv2'

In [2]:
from IPython.core.debugger import set_trace

import sys
sys.path.insert(0, "/home/saint/gazay/berloga-dl/lenin")

from lenin import train, test
from src.components.TernausNetV2 import *
from torchbearer.callbacks.checkpointers import Best

import torch
from torch import nn
from torch.nn import functional as F
from collections import OrderedDict

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [3]:
import string
import random
from time import gmtime, strftime

def id_generator(model_name):
    ts = strftime("%d-%m-%Y_%H:%M:%S", gmtime())
    return ts + '_' + model_name

RANDOM_KEY = id_generator(MODEL_NAME)
print(RANDOM_KEY)

20-08-2018_20:49:59_ternausv2


In [5]:
from lenin.datasets.salt import Dataset


from skimage.transform import resize
import albumentations as aug
pad = aug.PadIfNeeded(min_height=128, min_width=128)
# Temp preprocessing
def upsample(img):
    return resize(img, (128, 128), mode='constant', preserve_range=True)

# Temp preprocessing
def downsample(img):
    return resize(img, (101, 101), mode='constant', preserve_range=True)

def img_preprocess(img):
    #img = upsample(img)
    img = pad(image=img)['image']
    return img / 255.

def mask_preprocess(mask):
    #mask = upsample(mask)
    mask = pad(image=mask)['image']
    ones = np.expand_dims(mask, 2) > 0
    zeros = ones != True
    return np.concatenate((ones, zeros), axis=2)
    

dataset = Dataset(DIRECTORY, preprocessors={'image': img_preprocess, 'mask': mask_preprocess})
#dataset.check_integrity()

In [6]:
if False:
    pad = aug.PadIfNeeded(min_height=128, min_width=128)
    img = imread(DIRECTORY + '/train/masks/182ea1798b.png')
    ones = np.expand_dims(img, 2)
    zeros = ones == 0
    ones = zeros != 0
    mask = np.concatenate((ones, zeros), axis=2)
    print(mask.shape)
    pad(image=mask)['image'].shape    
if False:
    pad = aug.PadIfNeeded(min_height=128, min_width=128)
    img = imread(DIRECTORY + '/train/images/182ea1798b.png') / 255.
    #img[:, :, 1] = np.cumsum(img[:, :, 0], axis=0)
    img[:, :, 1] = np.cumsum(img[:, :, 0], axis=0)
#from skimage.transform import resize
#resize()
if False:
    img = dataset.image('train-182ea1798b')
    plt.figure(figsize=(15,10))
    plt.subplot(1, 4, 1)
    plt.imshow(np.transpose(img, (1, 2, 0)))
    plt.subplot(1, 4, 2)
    plt.imshow(dataset.downsample(np.transpose(img, (1, 2, 0))))

In [7]:
class TernausNetV2(nn.Module):
    fields = ('image', 'mask')
    test_fields = ('image',)
    
    def __init__(self, num_classes=2,
                       num_filters=32,
                       is_deconv=False,
                       num_input_channels=11):
        super().__init__()
        conf = {
             "network": {
                "arch": "wider_resnet38",
                "activation": "leaky_relu",
                "leaky_relu_slope": 0.01,
                "input_3x3": True,
                "bn_mode": "inplace",
                "classes": 1000
            }
        }
        
#         freeze = False

        self.pool = nn.MaxPool2d(2, 2)

        model_params = get_model_params(conf["network"])

        encoder = WiderResNet(structure=[3, 3, 6, 3, 1, 1], **model_params)

        self.conv1 = nn.Sequential(
            OrderedDict([('conv1', nn.Conv2d(num_input_channels, 64, 3, padding=1, bias=False))]))
        self.conv2 = encoder.mod2
        self.conv3 = encoder.mod3
#         if freeze:
#             for param in self.conv3.parameters():
#                 param.requires_grad = False
        self.conv4 = encoder.mod4
#         if freeze:
#             for param in self.conv4.parameters():
#                 param.requires_grad = False
        self.conv5 = encoder.mod5
#         if freeze:
#             for param in self.conv5.parameters():
#                 param.requires_grad = False
        
        dec_size = 1024
        self.center = DecoderBlock(dec_size, num_filters * 8, num_filters * 8, is_deconv=is_deconv)
        self.dec5 = DecoderBlock(dec_size + num_filters * 8, num_filters * 8, num_filters * 8, is_deconv=is_deconv)
#         if freeze:
#             for param in self.dec5.parameters():
#                 param.requires_grad = False
        self.dec4 = DecoderBlock(dec_size//2 + num_filters * 8, num_filters * 8, num_filters * 8, is_deconv=is_deconv)
#         if freeze:
#             for param in self.dec4.parameters():
#                 param.requires_grad = False
        self.dec3 = DecoderBlock(dec_size//4 + num_filters * 8, num_filters * 2, num_filters * 2, is_deconv=is_deconv)
#         if freeze:
#             for param in self.dec3.parameters():
#                 param.requires_grad = False
        self.dec2 = DecoderBlock(dec_size//8 + num_filters * 2, num_filters * 2, num_filters, is_deconv=is_deconv)
        self.dec1 = ConvRelu(dec_size//16 + num_filters, num_filters)
        self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)
        
        
    def forward(self, batch):
        x = batch.image#['image']#
        conv1 = self.conv1(x)
        conv2 = self.conv2(self.pool(conv1))
        conv3 = self.conv3(self.pool(conv2))
        conv4 = self.conv4(self.pool(conv3))
        conv5 = self.conv5(self.pool(conv4))

        center = self.center(self.pool(conv5))

        dec5 = self.dec5(torch.cat([center, conv5], 1))

        dec4 = self.dec4(torch.cat([dec5, conv4], 1))
        dec3 = self.dec3(torch.cat([dec4, conv3], 1))
        dec2 = self.dec2(torch.cat([dec3, conv2], 1))
        dec1 = self.dec1(torch.cat([dec2, conv1], 1))
        final = self.final(dec1)
        return torch.sigmoid(final)

In [8]:
def get_model(num_input_channels):
    model = TernausNetV2(num_classes=2)
    state = torch.load(WEIGHTS_PATH)
    state = {key.replace('module.', ''): value for key, value in state['model'].items()}

    model.load_state_dict(state)
    model.eval()
    
    model.conv1 = nn.Sequential(
        nn.Conv2d(num_input_channels, 11, 1, padding=0, bias=False), model.conv1)
    
    #model.final = nn.Sequential(
    #    model.final, nn.Conv2d(1, 1, kernel_size=3))

    if torch.cuda.is_available():
        model.cuda()
    return model

def load_model(model_path):
    model = get_model(3)
    state = torch.load(model_path)
    model.load_state_dict(state['model'])
    model.eval()
    return model

In [9]:
TRAIN = True#False#
if TRAIN:
    model = get_model(3)

In [10]:
if TRAIN:
    save_filepath=SAVE_PATH + ('%s.{epoch:02d}-{val_loss:.4f}.pt' % RANDOM_KEY)
    checkpointer = Best(filepath=save_filepath)
    train(model, dataset,
          augment={ ('image', 'mask'): [
              #{'type': 'HorizontalFlip'},
              #{'type': 'Blur'},
              #{'type': 'RandomCrop', 'height': 128, 'width': 128}
          ] },
          batch_size=30,
          optimizer={ 'type': 'Adam', 'lr': 1e-4 },
          epochs=15,
          loss={ 'type': 'BCELoss' },
          metrics=['loss'],
          callbacks=[checkpointer])


/usr/local/lib/python3.5/dist-packages/torchbearer/torchbearer.py:27: UserWarning: The Model class and all of its methods will be deprecated in the next version (0.2.0) in favor of the upcoming Trial API
  warnings.warn('The Model class and all of its methods will be deprecated in the next version (0.2.0) in favor of the upcoming Trial API')
0/15(t):   0%|          | 0/100 [00:00<?, ?it/s]

ValueError: Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/saint/gazay/berloga-dl/lenin/lenin/preloader/dataloader.py", line 9, in collate_fn
    values = [getattr(dataset, name)(record, augmentors[name]) for record in records]
  File "/home/saint/gazay/berloga-dl/lenin/lenin/preloader/dataloader.py", line 9, in <listcomp>
    values = [getattr(dataset, name)(record, augmentors[name]) for record in records]
  File "/home/saint/gazay/berloga-dl/lenin/lenin/datasets/salt.py", line 70, in mask
    return np.transpose(mask, (2, 0, 1)).astype('float32')
  File "/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py", line 575, in transpose
    return _wrapfunc(a, 'transpose', axes)
  File "/usr/local/lib/python3.5/dist-packages/numpy/core/fromnumeric.py", line 52, in _wrapfunc
    return getattr(obj, method)(*args

In [ ]:
!ls {SAVE_PATH}

# INFERENCE

In [ ]:
if not TRAIN:
    model_name = '19-08-2018_09:09:51_ternausv2.06-0.1506.pt'
    model = load_model(SAVE_PATH + model_name)

ids= ['1f1cc6b3a4','5b7c160d0d','6c40978ddf'] #,'7dfdf6eeb8','7e5a6e5013']
def show(ids):
    for j, img_name in enumerate(ids):
        q = j + 1
        img = cv2.imread(DIRECTORY + '/train/images/' + img_name + '.png')
        img_mask = cv2.imread(DIRECTORY + '/train/masks/' + img_name + '.png')

        plt.figure(figsize=(15,10))
        plt.subplot(1, 2, 1)
        plt.imshow(img)
        plt.subplot(1, 2, 2)
        plt.imshow(img_mask)

depths = read_csv(DIRECTORY + '/depths.csv', index_col=0)['z']

img_id = '5b7c160d0d'
orig_img = imread(DIRECTORY + '/train/images/' + img_id +'.png')
orig_mask = imread(DIRECTORY + '/train/masks/' + img_id + '.png')
pad = aug.PadIfNeeded(min_height=128, min_width=128)
crop = aug.CenterCrop(height=101, width=101)
img = pad(image=orig_img)["image"] / 255.

#img[:, :, 1] = np.cumsum(img[:, :, 0], axis=0)
#img[:, :, 2] = depths[img_id] / 1000.

img = np.transpose(img, (2, 0, 1)).astype('float32') 
img = np.expand_dims(img, 0)
#print(img.shape)
img = torch.from_numpy(img)

class Batch(object):
    pass
batch = Batch()
batch.image = torch.autograd.Variable(img).cuda()
pred = model(batch)
#pred = model({ 'image': batch.image })
#pred

In [ ]:
mask = pad(image=orig_mask)["image"]
ones = np.expand_dims(mask, 2) > 0
zeros = ones != True
#ones = !zeros
mask = np.concatenate((ones, zeros), axis=2)
mask[:, :, 0].astype('float32')

In [ ]:
y_pred = pred.data[0][0].cpu().numpy()
import matplotlib.pyplot as plt
%matplotlib inline
print(mask.shape)
plt.figure(figsize=(15,10))
plt.subplot(1, 4, 1)
plt.imshow(orig_img)
plt.subplot(1, 4, 2)
plt.imshow(orig_mask)
plt.subplot(1, 4, 3)
plt.imshow(crop(image=y_pred)['image'])
plt.subplot(1, 4, 4)
plt.imshow(crop(image=mask[:,:,1])['image'])

In [ ]:
from torchbearer import Model
from lenin.augmentors.image import Augmentor

def predict(model, dataset, **options):
    batch_size = options.pop('batch_size', 32)

    augmentors = {}
    for fields, augment in options.pop('augment', {}).items():
        if isinstance(fields, str):
            fields = [fields]
        augmentor = Augmentor(augment)
        for field in fields:
            augmentors[field] = augmentor

    preload_opts = options.pop('preloader', { 'shuffle': False })
    testgen = __preloader_predict(model, dataset, 'test_fields', dataset.test, batch_size, preload_opts, augmentors)

    device = options.pop('device', 'cuda')
    bearer = Model(model, torch.optim.Adam(model.parameters())).to(device)
    return bearer.predict_generator(testgen)


def __preloader_predict(model, dataset, fields_name, records, batch_size, preload_opts, augmentors={}):
    fields = __fields(model, dataset, fields_name)
    def collate_fn(records):
        batch = { 'records': torch.utils.data.dataloader.default_collate(records) }
        for name in fields:
            if augmentors.get(name):
                values = [getattr(dataset, name)(record, augmentors[name]) for record in records]
            else:
                values = [getattr(dataset, name)(record) for record in records]
            batch[name] = torch.utils.data.dataloader.default_collate(values)
        return batch
    return torch.utils.data.DataLoader(records, batch_size, collate_fn=collate_fn, **preload_opts)


def __fields(model, dataset, fields_name):
    return getattr(model, fields_name, None) or getattr(model, 'fields', None) or dataset.fields

In [ ]:
predictions = test(model, dataset,
    augment={ ('image',): [{'type': 'PadIfNeeded', 'min_height': 128, 'min_width': 128}] }
)

In [ ]:
val_dataset = Dataset(DIRECTORY)
val_dataset.test = dataset.train[:2000]
val_predictions = test(model, val_dataset,
    augment={ ('image',): [{'type': 'PadIfNeeded', 'min_height': 128, 'min_width': 128}] }
)

In [ ]:
_img = val_predictions[1803].data[0].cpu().numpy()
plt.figure(figsize=(15,10))
plt.subplot(1, 4, 1)
plt.imshow(_img)

In [ ]:
val_preds = np.array([crop(image=pred)['image'] for pred in val_predictions.data.cpu().numpy()[:, 0, :, :]])
val_masks = [crop(image=val_dataset.mask(rec).numpy()[0])['image'] for rec in val_dataset.test]

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(1, 4, 1)
plt.imshow(val_preds[1803])
plt.subplot(1, 4, 2)
plt.imshow(val_masks[1803])

In [ ]:
from sklearn.metrics import jaccard_similarity_score

metric_by_threshold = []
for threshold in np.linspace(0, 1, 11):
    val_binary_prediction = (val_preds > threshold).astype(int)
    
    iou_values = []
    for y_mask, p_mask in zip(val_masks, val_binary_prediction):
        iou = jaccard_similarity_score(y_mask.flatten(), p_mask.flatten())
        iou_values.append(iou)
    iou_values = np.array(iou_values)
    
    accuracies = [
        np.mean(iou_values > iou_threshold)
        for iou_threshold in np.linspace(0.5, 0.95, 10)
    ]
    print('Threshold: %.1f, Metric: %.3f' % (threshold, np.mean(accuracies)))
    metric_by_threshold.append((np.mean(accuracies), threshold))
    
best_metric, best_threshold = max(metric_by_threshold)

In [ ]:
predictions = np.array([crop(image=pred)['image'] for pred in predictions.data.cpu().numpy()[:, 0, :, :]])
binary_predictions = (predictions > best_threshold).astype(int)

def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b > prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

all_masks = []
for p_mask in list(binary_predictions):
    p_mask = rle_encoding(p_mask)
    all_masks.append(' '.join(map(str, p_mask)))

In [ ]:
import pandas as pd
submit = pd.DataFrame([[rec.split('-')[1] for rec in dataset.test], all_masks]).T
submit.columns = ['id', 'rle_mask']
submit.to_csv('/home/saint/submissions/salt/%s.csv' % model_name, index = False)

In [ ]:
submit = SUBMISSIONS_PATH + model_name + '.csv'
!kaggle c submit -f {submit.replace(':', '\:')} -m '{submit}' -c tgs-salt-identification-challenge